In [2]:
%load_ext autoreload
%autoreload 2
from misc import project_utils

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
# tf.get_logger().setLevel('INFO')
# tf.autograph.set_verbosity(1)
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    print('Invalid device. Error.')
    pass

from tensorflow import keras
from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

from random import randint

RANDOM_SEED = 0
BATCH_SIZE = 32
EPOCHS = 100
LEARNING_RATE = 0.0001
ES_PATIENCE = 10
ES_RESTORE_WEIGHTS = True
PLAY_VIDEOS = False

In [2]:
df_train = project_utils.load_image_dataframes('./data/train/', 'annotations.json')
display(df_train)

,id,confidence,y1,x1,y2,x2
0,00001.jpg,1.0,152.0,140.0,397.0,346.0
1,00002.jpg,1.0,144.0,140.0,403.0,345.0
2,00003.jpg,1.0,142.0,138.0,402.0,346.0
3,00004.jpg,1.0,144.0,140.0,405.0,344.0
4,00005.jpg,1.0,141.0,140.0,407.0,344.0
...,...,...,...,...,...,...
936,00937.jpg,0.0,0.0,0.0,0.0,0.0
937,00938.jpg,0.0,0.0,0.0,0.0,0.0
938,00939.jpg,0.0,0.0,0.0,0.0,0.0
939,00940.jpg,0.0,0.0,0.0,0.0,0.0


In [3]:
df_test = project_utils.load_image_dataframes('./data/test/', 'annotations.json')
display(df_test)

,id,confidence,y1,x1,y2,x2
0,00001.jpg,1.0,220.0,252.0,411.0,437.0
1,00002.jpg,1.0,217.0,241.0,402.0,432.0
2,00003.jpg,1.0,229.0,252.0,416.0,443.0
3,00004.jpg,1.0,243.0,317.0,425.0,487.0
4,00005.jpg,1.0,274.0,332.0,457.0,510.0
...,...,...,...,...,...,...
107,00108.jpg,0.0,0.0,0.0,0.0,0.0
108,00109.jpg,0.0,0.0,0.0,0.0,0.0
109,00110.jpg,0.0,0.0,0.0,0.0,0.0
110,00111.jpg,0.0,0.0,0.0,0.0,0.0


In [4]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train, 
    directory='./data/train/', 
    x_col='id', 
    y_col=['confidence', 'y1', 'x1', 'y2', 'x2'], 
    subset='training', 
    batch_size=BATCH_SIZE, 
    seed=RANDOM_SEED, 
    shuffle=True, 
    class_mode='raw', 
    target_size=(224, 224), 
    color_mode='rgb')

validation_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train, 
    directory='./data/train/', 
    x_col='id', 
    y_col=['confidence', 'y1', 'x1', 'y2', 'x2'], 
    subset='validation', 
    batch_size=BATCH_SIZE, 
    seed=RANDOM_SEED, 
    shuffle=True, 
    class_mode='raw', 
    target_size=(224, 224), 
    color_mode='rgb')

Found 753 validated image filenames.
Found 188 validated image filenames.


In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_test, 
    directory='./data/test/', 
    x_col='id', 
    y_col=['confidence', 'y1', 'x1', 'y2', 'x2'], 
    batch_size=BATCH_SIZE, 
    seed=RANDOM_SEED, 
    shuffle=True, 
    class_mode='raw', 
    target_size=(224, 224), 
    color_mode='rgb')

Found 112 validated image filenames.


Epoch 1/100
24/24 [==============================] - 39s 1s/step - loss: 111445.3831 - iou_metric: 0.0000e+00 - val_loss: 73051.1562 - val_iou_metric: 0.0000e+00
Epoch 2/100
24/24 [==============================] - 6s 237ms/step - loss: 79921.7603 - iou_metric: 7.5554e-04 - val_loss: 37839.2109 - val_iou_metric: 0.0174
Epoch 3/100
24/24 [==============================] - 6s 237ms/step - loss: 50053.8634 - iou_metric: 0.0392 - val_loss: 16324.9219 - val_iou_metric: 0.1244
Epoch 4/100
24/24 [==============================] - 6s 237ms/step - loss: 34454.2633 - iou_metric: 0.1229 - val_loss: 9411.1113 - val_iou_metric: 0.1682
Epoch 5/100
24/24 [==============================] - 6s 237ms/step - loss: 31461.6615 - iou_metric: 0.1390 - val_loss: 8018.5640 - val_iou_metric: 0.2002
Epoch 6/100
24/24 [==============================] - 6s 237ms/step - loss: 29284.1983 - iou_metric: 0.1633 - val_loss: 7591.2817 - val_iou_metric: 0.2155
Epoch 7/100
24/24 [==============================] - 6s 238ms/

4/4 [==============================] - 11s 3s/step - loss: 24421.1113 - iou_metric: 0.0956
Training Loss: 977.3382568359375
Training IOU: 0.5736852288246155
Validation Loss: 5363.9052734375
Validation IOU: 0.4347985088825226
Testing Loss: 24421.111328125
Testing IOU: 0.0955590233206749


In [6]:
vgg = keras.applications.VGG19(weights='imagenet', include_top=False, input_tensor=keras.layers.Input(shape=(224, 224, 3)))
vgg.trainable = False

flatten = vgg.output
flatten = keras.layers.Flatten()(flatten)
bbHead = keras.layers.Dense(1024, activation='relu')(flatten)

bbHead = keras.layers.Dense(5, activation='linear', name='bounding_output')(bbHead)



model = keras.models.Model(inputs=vgg.input, outputs=bbHead)
opt = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model.compile(loss='mse', optimizer=opt, metrics=[project_utils.iou_metric])

es = EarlyStopping(monitor='val_iou_metric', mode='max', verbose=1, patience=ES_PATIENCE, restore_best_weights=ES_RESTORE_WEIGHTS)
history = model.fit(train_generator, validation_data=validation_generator, epochs=EPOCHS, callbacks=[es])
project_utils.plot_history(history)

train_results = model.evaluate(train_generator)
validation_results = model.evaluate(validation_generator)
test_results  = model.evaluate(test_generator)
print('Training Loss: {}\nTraining IOU: {}'.format(train_results[0], train_results[1]))
print('Validation Loss: {}\nValidation IOU: {}'.format(validation_results[0], validation_results[1]))
print('Testing Loss: {}\nTesting IOU: {}'.format(test_results[0], test_results[1]))

project_utils.write_results('./history.txt', model, history, train_generator.n, validation_generator.n)

In [7]:
if PLAY_VIDEOS:
    display(project_utils.play_images_from_datagen(train_generator, model=model, scale_down=True, threshold=0.25, num_to_iter=100))

In [3]:
if PLAY_VIDEOS:
    display(project_utils.play_images_from_datagen(validation_generator, model=model, scale_down=True, threshold=0.25, num_to_iter=50))

In [9]:
if PLAY_VIDEOS:
    display(project_utils.play_images_from_datagen(test_generator, model=model, scale_down=True, threshold=0.25))

In [10]:
if PLAY_VIDEOS:
    display(project_utils.play_images_from_video('../../data/original/test_videos/hand_signs_webcam.mp4', model=model, threshold=0.25))

In [11]:
if PLAY_VIDEOS:
    display(project_utils.play_images_from_video('../../data/original/test_videos/hand_signs_hand_only.mp4', model=model, threshold=0.25))